In [42]:
##########################################
#Examples for different configurations:
#
# Assume example data set:
# - Patient1
#   - pat1_TP1_MR
#   - pat1_TP2_MR
#   - pat1_TP1_CT
# - Patient2
#   - pat2_TP1_MR1
#   - pat2_TP1_MR2
#   - pat2_TP2_MR1
#   - pat2_TP1_CT
#   - pat2_TP2_CT

In [43]:
###############################################################################
# Imports
###############################################################################
import os

import avid.common.workflow as workflow
import avid.common.artefact.defaultProps as artefactProps

from avid.selectors import ActionTagSelector as ATS
from avid.actions.pythonAction import PythonNaryBatchAction as naryPythonAction, PythonNaryBatchActionV2

from avid.linkers import CaseLinker, TimePointLinker, FractionLinker
from avid.splitter import SingleSplitter, CaseSplitter, KeyValueSplitter

In [44]:
###############################################################################
# Define ActionTagSelectors
###############################################################################
mr_image_selector = ATS('MR')
ct_image_selector = ATS('CT')
mask_selector = ATS('Seg')

In [45]:
###############################################################################
# Define callable
###############################################################################
def my_function(outputs, **kwargs):
    '''
        Simple print-callable that outputs the combinations of input artifacts.
    '''
    keys = kwargs.keys()
    print_output = 'output: {'
    
    for i, key in enumerate(keys):
        if i != 0:
            print_output += ', '
        print_output += '{}:{}'.format(key, [os.path.basename(mr) for mr in kwargs[key]])
    print_output += '}'
    
    print(print_output)
    with open(outputs[0], "w") as ofile:
        ofile.write(str(kwargs))

In [46]:
###############################################################################
# Initialize session
###############################################################################
session =  workflow.initSession(bootstrapArtefacts=os.path.join(os.getcwd(),'output', 'example.avid'),
                                sessionPath=os.path.join(os.getcwd(),'output', 'example'),
                                expandPaths=True,
                                debug=True)

In [47]:
###############################################################################
# Example 1: only select a specific modality (here MR) and make a call per image
###############################################################################
with session:
    PythonNaryBatchActionV2(primaryInputSelector=mr_image_selector, 
                        actionTag="example_1", 
                        generateCallable=my_function,
                        passOnlyURLs=True).do()

2021-12-09 09:57:23,463 [INFO] Starting action: PythonNaryBatchActionV2_example_1 (UID: 70c950c6-4940-4857-a59b-b477af013f73) ...
2021-12-09 09:57:23,465 [INFO] Starting action: my_function (UID: b9c9a545-5f64-4fed-b80b-15160e1ab4ee) ...
output: {primaryInput:['pat1_TP1_MR.txt']}
2021-12-09 09:57:23,470 [INFO] Finished action: my_function (UID: b9c9a545-5f64-4fed-b80b-15160e1ab4ee) -> SUCCESS
2021-12-09 09:57:23,471 [INFO] Starting action: my_function (UID: dca42d61-7a53-4b2e-850d-bbacd4ddf4b2) ...
output: {primaryInput:['pat1_TP2_MR.txt']}
2021-12-09 09:57:23,476 [INFO] Finished action: my_function (UID: dca42d61-7a53-4b2e-850d-bbacd4ddf4b2) -> SUCCESS
2021-12-09 09:57:23,477 [INFO] Starting action: my_function (UID: f8d0d7fe-4175-4168-b916-6bc1a75454ba) ...
output: {primaryInput:['pat2_TP1_MR1.txt']}
2021-12-09 09:57:23,480 [INFO] Finished action: my_function (UID: f8d0d7fe-4175-4168-b916-6bc1a75454ba) -> SUCCESS
2021-12-09 09:57:23,481 [INFO] Starting action: my_function (UID: b471e

In [48]:
###############################################################################
#Example 2: like example 1 but choose another variable name
###############################################################################
with session:
    PythonNaryBatchActionV2(primaryInputSelector=mr_image_selector,
                        primaryAlias='mr_images',
                        actionTag="example_2", 
                        generateCallable=my_function, 
                        passOnlyURLs=True).do()

2021-12-09 09:57:23,509 [INFO] Starting action: PythonNaryBatchActionV2_example_2 (UID: d4df9f12-9211-438b-86d7-610ace6f2076) ...
2021-12-09 09:57:23,512 [INFO] Starting action: my_function (UID: 2ece8f04-01fb-4cfb-a1dc-f6361bf8e782) ...
output: {mr_images:['pat1_TP1_MR.txt']}
2021-12-09 09:57:23,516 [INFO] Finished action: my_function (UID: 2ece8f04-01fb-4cfb-a1dc-f6361bf8e782) -> SUCCESS
2021-12-09 09:57:23,517 [INFO] Starting action: my_function (UID: 6bfd3701-e637-4d55-a224-9fab724de41b) ...
output: {mr_images:['pat1_TP2_MR.txt']}
2021-12-09 09:57:23,521 [INFO] Finished action: my_function (UID: 6bfd3701-e637-4d55-a224-9fab724de41b) -> SUCCESS
2021-12-09 09:57:23,522 [INFO] Starting action: my_function (UID: 46dd31b3-da1e-4a14-b2cc-b73b62d4bc94) ...
output: {mr_images:['pat2_TP1_MR1.txt']}
2021-12-09 09:57:23,525 [INFO] Finished action: my_function (UID: 46dd31b3-da1e-4a14-b2cc-b73b62d4bc94) -> SUCCESS
2021-12-09 09:57:23,526 [INFO] Starting action: my_function (UID: f5827dd0-e429-

In [49]:
###############################################################################
#Example 3: select MR and CT images and call them pairwise (folding both sets per case)
###############################################################################
with session:
    PythonNaryBatchActionV2(primaryInputSelector=mr_image_selector,
                        primaryAlias='mr_images',
                        additionalInputSelectors = {'ct_images': ct_image_selector},
                        actionTag="example_3",
                        generateCallable=my_function,
                        passOnlyURLs=True).do()

2021-12-09 09:57:23,557 [INFO] Starting action: PythonNaryBatchActionV2_example_3 (UID: 27ef1e2d-dd0b-47d1-bd37-eb9ac10ffaa7) ...
2021-12-09 09:57:23,559 [INFO] Starting action: my_function (UID: d1df84eb-e535-46d1-b83f-8cbdadedb15c) ...
output: {mr_images:['pat1_TP1_MR.txt'], ct_images:['pat1_TP1_CT.txt']}
2021-12-09 09:57:23,564 [INFO] Finished action: my_function (UID: d1df84eb-e535-46d1-b83f-8cbdadedb15c) -> SUCCESS
2021-12-09 09:57:23,565 [INFO] Starting action: my_function (UID: 7a3a9985-ece3-42eb-9004-ea2b8f0caac6) ...
output: {mr_images:['pat1_TP2_MR.txt'], ct_images:['pat1_TP1_CT.txt']}
2021-12-09 09:57:23,570 [INFO] Finished action: my_function (UID: 7a3a9985-ece3-42eb-9004-ea2b8f0caac6) -> SUCCESS
2021-12-09 09:57:23,571 [INFO] Starting action: my_function (UID: b441b692-9a47-4c46-b331-526abc098e48) ...
output: {mr_images:['pat2_TP1_MR1.txt'], ct_images:['pat2_TP1_CT.txt']}
2021-12-09 09:57:23,575 [INFO] Finished action: my_function (UID: b441b692-9a47-4c46-b331-526abc098e48

In [50]:
###############################################################################
#Example 3b:  select MR and CT images and call them pairwise (but only MRs and CTs of same patient and timepoint)
###############################################################################
with session:
    splitProperties=[artefactProps.CASE]
    PythonNaryBatchActionV2(primaryInputSelector=mr_image_selector,
                            primaryAlias='mr_images',
                            additionalInputSelectors = {'additional': ct_image_selector},
                            linker={'additional': CaseLinker()+TimePointLinker()},
                            actionTag="example_3b",
                            generateCallable=my_function,
                            passOnlyURLs=True).do()

2021-12-09 09:57:23,620 [INFO] Starting action: PythonNaryBatchActionV2_example_3b (UID: 87e29be4-ed8c-45c6-b075-32443249821a) ...
2021-12-09 09:57:23,623 [INFO] Starting action: my_function (UID: a314e0d7-e0ed-47b3-b41c-07e703dba563) ...
output: {mr_images:['pat1_TP1_MR.txt'], additional:['pat1_TP1_CT.txt']}
2021-12-09 09:57:23,628 [INFO] Finished action: my_function (UID: a314e0d7-e0ed-47b3-b41c-07e703dba563) -> SUCCESS
2021-12-09 09:57:23,629 [INFO] Starting action: my_function (UID: 2e6f30ca-fe44-4c8a-82f7-3bb47e84b4ca) ...
output: {mr_images:['pat2_TP1_MR1.txt'], additional:['pat2_TP1_CT.txt']}
2021-12-09 09:57:23,635 [INFO] Finished action: my_function (UID: 2e6f30ca-fe44-4c8a-82f7-3bb47e84b4ca) -> SUCCESS
2021-12-09 09:57:23,636 [INFO] Starting action: my_function (UID: 42a20343-d3ae-4b19-890f-fa970755079d) ...
output: {mr_images:['pat2_TP1_MR2.txt'], additional:['pat2_TP1_CT.txt']}
2021-12-09 09:57:23,640 [INFO] Finished action: my_function (UID: 42a20343-d3ae-4b19-890f-fa97075

In [51]:
###############################################################################
#Example 3c:  select MR and CT images and call them blockwise (but only MRs and CTs of same patient)
###############################################################################
with session:
    splitProperties=[artefactProps.CASE]
    PythonNaryBatchActionV2(primaryInputSelector=mr_image_selector,
                        primaryAlias='mr_images',
                        additionalInputSelectors = {'additional': ct_image_selector},
                        linker={'additional': CaseLinker(allowOnlyFullLinkage=False)+TimePointLinker(allowOnlyFullLinkage=False)},
                        splitter={'mr_images': KeyValueSplitter(*splitProperties), 'additional': CaseSplitter()},
                        actionTag="example_3b", 
                        generateCallable=my_function,
                        passOnlyURLs=True).do()

2021-12-09 09:57:23,665 [INFO] Starting action: PythonNaryBatchActionV2_example_3b (UID: 799daa9a-5f07-4f55-994e-67aaa1872c5c) ...
2021-12-09 09:57:23,666 [INFO] Starting action: my_function (UID: b3970b9f-d2b2-4ee1-9f16-c182a0da3084) ...
output: {mr_images:['pat1_TP1_MR.txt', 'pat1_TP2_MR.txt'], additional:['pat1_TP1_CT.txt']}
2021-12-09 09:57:23,671 [INFO] Finished action: my_function (UID: b3970b9f-d2b2-4ee1-9f16-c182a0da3084) -> SUCCESS
2021-12-09 09:57:23,671 [INFO] Starting action: my_function (UID: 4e6a20d9-fcc0-467b-ad14-d4de5d461e94) ...
output: {mr_images:['pat2_TP1_MR1.txt', 'pat2_TP1_MR2.txt', 'pat2_TP2_MR1.txt'], additional:['pat2_TP1_CT.txt', 'pat2_TP2_CT.txt']}
2021-12-09 09:57:23,674 [INFO] Finished action: my_function (UID: 4e6a20d9-fcc0-467b-ad14-d4de5d461e94) -> SUCCESS
2021-12-09 09:57:23,675 [INFO] Finished action: PythonNaryBatchActionV2_example_3b (UID: 799daa9a-5f07-4f55-994e-67aaa1872c5c) -> SUCCESS
2021-12-09 09:57:23,676 [INFO] Successful actions: 4.
2021-12-

In [52]:
###############################################################################
#Example 4: select MR images and masks and call them block wise per case
###############################################################################
with session:
    PythonNaryBatchActionV2(primaryInputSelector=mr_image_selector,
                        primaryAlias='mr_images',
                        additionalInputSelectors ={'additional': mask_selector},
                        linker={'additional': CaseLinker(allowOnlyFullLinkage=False)+TimePointLinker(allowOnlyFullLinkage=False)},
                        splitter={'mr_images': CaseSplitter(), 'additional': CaseSplitter()},
                        actionTag="example_4", 
                        generateCallable=my_function, 
                        passOnlyURLs=True).do()

2021-12-09 09:57:23,696 [INFO] Starting action: PythonNaryBatchActionV2_example_4 (UID: 3850eee4-600a-46b7-ab35-01df9f3f818a) ...
2021-12-09 09:57:23,697 [INFO] Starting action: my_function (UID: f3d274f1-fb8c-4968-b269-22c15c5aa14c) ...
output: {mr_images:['pat1_TP1_MR.txt', 'pat1_TP2_MR.txt'], additional:['pat1_TP1_Seg1.txt', 'pat1_TP1_Seg2.txt', 'pat1_TP2_Seg1.txt']}
2021-12-09 09:57:23,702 [INFO] Finished action: my_function (UID: f3d274f1-fb8c-4968-b269-22c15c5aa14c) -> SUCCESS
2021-12-09 09:57:23,703 [INFO] Starting action: my_function (UID: d0178539-d213-4e6a-9fa9-42e928b0c5ae) ...
output: {mr_images:['pat2_TP1_MR1.txt', 'pat2_TP1_MR2.txt', 'pat2_TP2_MR1.txt'], additional:['pat2_TP1_Seg1.txt', 'pat2_TP2_Seg1.txt']}
2021-12-09 09:57:23,707 [INFO] Finished action: my_function (UID: d0178539-d213-4e6a-9fa9-42e928b0c5ae) -> SUCCESS
2021-12-09 09:57:23,708 [INFO] Starting action: my_function (UID: 5c5c1122-3ff9-4b33-8a0c-ec89159a304a) ...
output: {mr_images:['pat3_TP1_MR.txt'], addit